In [1]:
import os
import numpy as np
import pandas as pd
import sqlite3
import bz2
import csv

In [2]:
# Create the database
plane = sqlite3.connect('python_airline.db')

# cursor object to execute SQLite command/queries from Python.
c = plane.cursor()

In [3]:
ontime_2000 = pd.read_csv('../Datasets/2000.csv.bz2', encoding='latin-1')
ontime_2001 = pd.read_csv('../Datasets/2001.csv.bz2', encoding='latin-1')
ontime_2002 = pd.read_csv('../Datasets/2002.csv.bz2', encoding='latin-1')
ontime_2003 = pd.read_csv('../Datasets/2003.csv.bz2', encoding='latin-1')
ontime_2004 = pd.read_csv('../Datasets/2004.csv.bz2', encoding='latin-1')
ontime_2005 = pd.read_csv('../Datasets/2005.csv.bz2', encoding='latin-1')

In [7]:
ontime_1 = pd.concat([ontime_2000, ontime_2001], ignore_index=True)
ontime_2 = pd.concat([ontime_1, ontime_2002], ignore_index=True)
ontime_3 = pd.concat([ontime_2, ontime_2003], ignore_index=True)
ontime_4 = pd.concat([ontime_3, ontime_2004], ignore_index=True)
ontime = pd.concat([ontime_4, ontime_2005], ignore_index=True)

In [4]:
carrier_df = pd.read_csv('../Datasets/carriers.csv')
airport_df = pd.read_csv('../Datasets/airports.csv')
plane_df = pd.read_csv('../Datasets/plane-data.csv')

In [7]:
# Run it only once because if you run it again, it will replace your table.
# Transfer to the database
carrier_df.to_sql('carriers', con = plane, index = False)
airport_df.to_sql('airports', con = plane, index = False)
plane_df.to_sql('plane', con = plane, index = False)
ontime_2000.to_sql('ontime', con = plane, index = False)

#### Question 1

In [25]:
q1 = c.execute('''SELECT TailNum, AVG(DepDelay) as Ave_Delay FROM ontime  
                    WHERE Cancelled = 0  
                    AND Diverted = 0  
                    GROUP BY tailnum  
                    HAVING AVG(DepDelay) > 0  
                    ORDER BY Ave_Delay''').fetchall()
q1 = pd.DataFrame(q2)
q1.columns = ['TailNum','Average_Delay']
q1.to_csv('../python_sql/python_q1.csv', index=False)

#### Question 2

In [24]:
q2 = c.execute('''SELECT Dest, COUNT(Dest) AS number_of_In_Flights FROM ontime  
                WHERE Cancelled = 0  
                GROUP BY Dest  
                ORDER BY number_of_In_Flights DESC''').fetchall()
q2 = pd.DataFrame(q2)
q2.columns = ['States', 'Total_Flights']
q2.to_csv('../python_sql/python_q2.csv', index=False)

#### Question 3

In [23]:
q3 = c.execute('''SELECT o.UniqueCarrier, c.Description, sum(Cancelled) AS total_cancelled   
                FROM ontime o  
                LEFT JOIN carriers c ON o.UniqueCarrier = c.Code  
                GROUP BY o.UniqueCarrier  
                ORDER BY total_cancelled''').fetchall()
q3 = pd.DataFrame(q3)
q3.columns = ['Unique_Carrier','Description','Total_Cancelled_Flights']
q3.to_csv('../python_sql/python_q3.csv', index=False)

#### Question 4

In [26]:
q4 = c.execute('''SELECT UniqueCarrier, c.Description, count(UniqueCarrier) AS total_flights, sum(cancelled) AS total_cancelled_flights,  
            (CAST(count(UniqueCarrier) as FLOAT)/ CAST(sum(cancelled) AS FLOAT)) AS ratio 
            FROM ontime o 
            
            JOIN carriers c ON o.UniqueCarrier = c.Code 
            GROUP BY UniqueCarrier 
            ORDER BY ratio''').fetchall()
q4 = pd.DataFrame(q4)
q4.columns = ['Unique_Carrier','Description','Total_Flights','Total_Cancelled_Flights','Ratio']
q4.to_csv('../python_sql/python_q4.csv', index=False)